In [ ]:
#| label: qmtFig3jn

import scipy
import scipy.io as sio
from pathlib import Path
import numpy as np

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
data_dir = dataset_path / "06-MT-01-qMT" / "06-MT-01-qMT" 

numPulses = np.append(np.arange(10,100,10),np.arange(100,700,100))
numPulses = np.append(np.arange(1,6,1),numPulses)

dataSim_Pulses_mat =  scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/blochSim/dataSim_Pulses.mat')
datablochSim_Pulses_mat =  scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/blochSim/datablochSim_Pulses.mat')
dataRaw_Pulses_mat =  scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/blochSim/dataRaw_Pulses.mat')
datablochSimResetMz_Pulses_mat =  scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/blochSim/datablochSimResetMz_Pulses.mat')
dataRawResetMz_Pulses_mat =  scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/blochSim/dataRawResetMz_Pulses.mat')

dataSimAnalytical_Pulses = np.array(dataSim_Pulses_mat["dataSim_Pulses"])
dataBlochSim_Pulses = np.array(datablochSim_Pulses_mat["datablochSim_Pulses"])
dataRaw_Pulses = np.array(dataRaw_Pulses_mat["dataRaw_Pulses"])
dataBlochSimResetMz_Pulses = np.array(datablochSimResetMz_Pulses_mat["datablochSimResetMz_Pulses"])
dataRawResetMz_Pulses = np.array(dataRawResetMz_Pulses_mat["dataRawResetMz_Pulses"])



import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.image import imread
import scipy.io
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

# PYTHON CODE

init_notebook_mode(connected=True)

dataSimAnalyticalPulses1 = [dict(
        visible = False,
        x = dataSimAnalytical_Pulses[:,0,0],
        y = dataSimAnalytical_Pulses[:,1,ii],
        line = dict(color = "firebrick"),
        name = 'Analytical Solution (angle = 142)',
        hovertemplate = 'Analytical Solution (angle = 142)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataSimAnalyticalPulses1[4]['visible'] = True

dataSimAnalyticalPulses2 = [dict(
        visible = False,
        x = dataSimAnalytical_Pulses[:,0,0],
        y = dataSimAnalytical_Pulses[:,2,ii],
        line = dict(color = "royalblue"),
        name = 'Analytical Solution (angle = 426)',
        hovertemplate = 'Analytical Solution (angle = 426)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataSimAnalyticalPulses2[4]['visible'] = True

dataBlochSimPulses1 = [dict(
        visible = False,
        x = dataBlochSim_Pulses[:,0,0],
        y = dataBlochSim_Pulses[:,1,ii],
        line = dict(
            color = "firebrick",
            dash = 'dash'),
        name = 'Bloch Simulation (angle = 142)',
        text = 'Bloch Simulation (angle = 142)',
        hovertemplate = 'Bloch Simulation (angle = 142)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataBlochSimPulses1[4]['visible'] = True

dataBlochSimPulses2 = [dict(
        visible = False,
        x = dataBlochSim_Pulses[:,0,0],
        y = dataBlochSim_Pulses[:,2,ii],
        line = dict(
            color = "royalblue",
            dash = 'dash'),
        name = 'Bloch Simulation (angle = 426)',
        hovertemplate = 'Bloch Simulation (angle = 426)<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataBlochSimPulses2[4]['visible'] = True

dataRawPulses = [dict(
        visible = False,
        mode = 'markers',
        marker = dict(color = "darkslategray"),
        x = dataRaw_Pulses[:,0,0],
        y = dataRaw_Pulses[:,1,ii],
        name = 'Raw data',
        hovertemplate = 'Raw data<br>M<sub>z</sub> = %{y}<br>Offset = %{x} Hz<extra></extra>') for ii in range(len(numPulses))]

dataRawPulses[4]['visible'] = True

data = dataSimAnalyticalPulses1 + dataSimAnalyticalPulses2 + dataBlochSimPulses1 + dataBlochSimPulses2 + dataRawPulses

steps = []
for i in range(len(numPulses)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(dataSimAnalyticalPulses1)],
        label = str(numPulses[i])
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.0,
    active = 7,
    currentvalue = {"prefix": "# of Pulses: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    plot_bgcolor='rgba(0,0,0,0)',
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.2,
            showarrow=False,
            text='Frequency offset \u0394 (Hz)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.14,
            y=0.5,
            showarrow=False,
            text='Magnetization |M<sub>z</sub>|',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        type="log",
        range=[2, 5],
        dtick=1,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1.1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.05,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)